In [ ]:
from textblob import TextBlob
from bs4 import BeautifulSoup as BS
import  requests
import pandas as pd
from tqdm import tqdm
import time
from nltk.corpus import treebank_chunk 
from nltk.chunk import ne_chunk 
import spacy
from spacy import displacy
from collections import Counter 
nlp = spacy.load('en_core_web_sm')

In [ ]:
url="https://www.google.com/search?q=subsidiaries+of+Commonwealth+Bank+of+Australia"
page = requests.get(url)
soup = BS(page.content, 'html.parser')
results=soup.findAll(class_='am3QBf')
clean_results = [str(i).replace('</span>','').replace('</div>','')[65:] for i in results]
print(clean_results)

In [ ]:
# Looks for heading info
class Analysis:
    def __init__(self, term):
        self.term = term        
        self.url = "https://www.google.com/search?q="+(self.term)
    def run(self):
        response = requests.get(self.url)
        print(self.url)
        data = response.text
        soup = BS(data, 'html.parser')
        results=soup.findAll(class_='BNeawe s3v9rd AP7Wnd')
        headline = str(results[1])
        return headline.replace('<div class="BNeawe s3v9rd AP7Wnd">','')

In [ ]:
name = 'subsidiaries+of+bank+australia' 
Analysis(name).run()
doc = nlp('Retrieved Text DATA')
print([(X.text, X.label_) for X in doc.ents])

In [ ]:
# Looks for images info
class Analysis1:
    def __init__(self, term)
        self.term = term        
        self.url = "https://www.google.com/search?q="+(self.term)
    def run(self):
        response = requests.get(self.url)
        print(self.url)
        data = response.text
        soup = BS(data, 'html.parser')
        results = soup.findAll(class_='BNeawe s3v9rd AP7Wnd')
        headline = str(results).replace('</div','').split('<div class="BNeawe s3v9rd AP7Wnd">')
        return headline

In [ ]:
def crawl(name):
    a = Analysis(name) 
    result =  a.run()
    if len(result)>50:
        print('text code results:\n')
        print(result)
    else:
        a = Analysis1(name) 
        result =  a.run()
        print('pic code results:\n')
        subsidaries = []
        for i in result:
            if len(i) > 1:
                i = i.replace("...>, ","").replace('>,','').replace('.','').replace('(','').strip().lower()
                if i == '<div>':
                    break
                subsidaries.append(i)
        print(subsidaries)
        result = subsidaries.copy()
    return result

In [ ]:
name = 'subsidiaries+of+bank+australia' 
Analysis1(name)
crawl(name)

In [ ]:
data = pd.read_csv("data.csv")
n = data['Parent Company'].unique()
search_li = ['subsidiaries+of+'+i.replace(' ','+') for i in n]

In [ ]:
%%time
subsidary_li = []
for i in tqdm(search_li):
    try:
        subsidary_li.append(crawl(i))
        time.sleep(1)
    except:
        print('\ncouldnt do for ',i,'\n')

In [ ]:
df = pd.DataFrame({'company':n, 'subsidaries':subsidary_li})
out = pd.merge(data, df, left_on='Parent Name',right_on = 'company', how='left')

In [ ]:
out.to_csv('linked.csv')